In [54]:
import random
import sys
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState, DynamicFee
from model.amm.agents import Agent
from model.amm.trade_strategies import random_swaps, omnipool_arbitrage, constant_swaps, invest_all
from model.amm.global_state import GlobalState

# same seed, same parameters = same simulation result
random.seed(42)

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                'HDX': {'liquidity': 1000000, 'LRNA': 1000},
                'USD': {'liquidity': 10000, 'LRNA': 1000}
            },
            lrna_fee=DynamicFee(
                amplification=1,
                decay=0.00001,
                minimum=0.0005,
                maximum=0.1,
                current={'HDX': 0.01, 'USD': 0.01}
            ),
            asset_fee=0,
            lp_lrna_share=0,
            withdrawal_fee=False
        )
    },
    agents = {
        'LP': Agent(
            holdings={'HDX': 10000},
            trade_strategy=invest_all('omnipool')
        ),
        'Trader': Agent(
            holdings={'LRNA': float('inf')},
            trade_strategy=constant_swaps(
                pool_id='omnipool',
                sell_quantity=10000,
                sell_asset='LRNA',
                buy_asset='USD'
            )
        ),
        'Arbitrageur': Agent(
            holdings={'HDX': 100000000, 'USD': 1000000, 'LRNA': 100000},
            trade_strategy=omnipool_arbitrage(
                pool_id='omnipool',
            )
        )
    },
    external_market={'USD': 1, 'HDX': 0.01}
)

events = run.run(initial_state, time_steps=1)

Starting simulation...
Execution time: 0.005 seconds.


In [55]:
print(initial_state.cash_out('LP'))
print(events[-1].agents['LP'].holdings)

100.0
{'HDX': 0, 'LRNA': 0, 'USD': 0, ('omnipool', 'HDX'): 10000.0}


In [56]:
print('LRNA:', events[-1].pools['omnipool'].lrna)
print('Liquidity:', events[-1].pools['omnipool'].liquidity)


LRNA: {'HDX': 6034.875621890544, 'USD': mpf('5924.9258706467663')}
Liquidity: {'HDX': 169034.13821815164, 'USD': mpf('1687.78482943423')}


In [53]:
print('LRNA:', events[-1].pools['omnipool'].lrna)
print('Liquidity:', events[-1].pools['omnipool'].liquidity)


LRNA: {'HDX': 6034.875621890544, 'USD': mpf('5472.6870646766165')}
Liquidity: {'HDX': 169034.13821815164, 'USD': mpf('1827.255949740827')}


In [49]:
omnipool = events[-1].pools['omnipool']
print(omnipool.usd_price('HDX'))

0.0101701663790628


In [19]:
omnipool.lrna_fee('USD')

mpf('0.099980000000000013')